# Tokens and Embeddings


## 1. Tokenization

A text prompt sent to a model first needs to be broken down into tokens. The numerical token IDs are passed to the model. 

The tokenization scheme is designed hand-in-hand with the model, so the two are coupled: each model has a corresponding tokenizer which transforms the prompt text into the expect numerical representation.

### 1.1. Tokenization in Action
We can load a model to see what tokenization looks like practice.

We'll load a smaller open-source model and its corresponding tokenizer.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

if torch.cuda.is_available():
    device = "cuda"
    device_map = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
    device_map=None
else:
    device = "cpu"
    device_map = "cpu"

#### Load the model

This can take a few minutes depedning on internet connection.

In [6]:
model_name = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    torch_dtype="auto",
    trust_remote_code=True
)
model.to(device)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

#### Load the tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Use the model to generate text

First we tokenize the input promt. Then we pass this to the model.
We can peek in at each step to see what's actually being passed around.

We'll start with the following input prompt:

In [11]:
input_prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"

The tokenizer converts this text to a list of integers. These are the input IDs that are passed to the model.

In [ ]:
# Tokenize the input prompt 
input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids.to(device)

In [20]:
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001]], device='mps:0')


We can "decode" these input IDs, converting them back to text, to see how the tokenizer splits the text. It uses sub-word tokens, so `mishap` is split into `m, ish, ap`. Punctuation is its own token and there is a special token for `<|assistant|>` Spaces are implicit; parital tokens have a special hidden character preceding them and tokens without that character are assumed to have a space before them.

In [18]:
for id in input_ids[0]: 
    print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
the
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happened
.
<|assistant|>


We can now pass this tokenized input to the model to generate new tokens.

In [26]:
# Due to a quirk of Macs, we need to explicitly pass it an attention mask as it cannot be inferred
if device == 'mps':
    model_kwargs = {'attention_mask': (input_ids != tokenizer.pad_token_id).long()}
else:
    model_kwargs = {}

# Generate the text 
generation_output = model.generate(input_ids=input_ids, max_new_tokens=100, **model_kwargs)



The output of the generation appends tokens to the input.

In [27]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001,  3323,   622, 29901, 17778, 29888,  2152,
          6225, 11763,   363,   278, 19906,   292,   341,   728,   481,    13,
            13,    13, 29928,   799, 19235, 29892,    13,    13,    13, 29902,
          4966,   445,  2643, 14061,   366,  1532, 29889,   306,   626,  5007,
           304,  4653,   590,  6483,   342,  3095, 11763,   363,   278,   443,
          6477,   403, 15134,   393, 10761,   297,   596, 16423, 22600, 29889,
            13,    13,    13,  2887,   366,  1073, 29892,   306,   505,  2337,
          7336,  2859,   278, 15409,   322, 22024,   339,  1793,   310,   596,
         16423, 29889,   739,   756,  1063,   263,  2752,   310,  8681, 12232,
           363,   592, 29892,   322,   306,   471,  1468, 24455,   304,   505,
           278, 15130,   304,  1371]], device='mps:0

Again, we can decode this to see the output text

In [28]:
# Print the output 
print(tokenizer.decode(generation_output[0]))

Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|> Subject: Heartfelt Apologies for the Gardening Mishap


Dear Sarah,


I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in your garden yesterday.


As you know, I have always admired the beauty and tranquility of your garden. It has been a source of inspiration for me, and I was thrilled to have the opportunity to help


### 1.3. Tokenizer Design

There are three primary decisions that determine how the tokenizer splits a given prompt:
1. **Tokenization method**: byte pair encoding (BPE), WordPiece
2. **Tokenizer parameters**: vocabulary size, choice of special tokens
3. **Training data set**: a tokenizer trained on English text will give different results to one trained on Punjabi text or python code, etc.